In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%autoreload

import os
import sys
import numpy as np
sys.path.append('./src-py')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import datasets

### Preparing the Data:

Commands to prepare the datasets for llama3 and Qwen

In [ ]:
python prepare_dataset.py /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned/ /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-preprocessed-for-llama3/ meta-llama/Meta-Llama-3-8B-Instruct --max_num_turns=40

In [ ]:
python prepare_dataset.py /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-cleaned/ /mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-preprocessed-for-llama3/ meta-llama/Meta-Llama-3-8B-Instruct --max_num_turns=40

In [ ]:
prepare_dataset.py /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned/ /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-preprocessed-for-qwen/ Qwen/Qwen2.5-7B-Instruct --max_num_turns=30

-----------

In [11]:
train_researcher_dataset = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-preprocessed-for-llama3/train_researcher_ds/')

In [12]:
train_researcher_dataset

DatasetDict({
    train: Dataset({
        features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion'],
        num_rows: 97849
    })
    test: Dataset({
        features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion'],
        num_rows: 10720
    })
})

In [13]:
print(train_researcher_dataset['train']['prompt'][1])
print(train_researcher_dataset['train']['completion'][1])

[{'content': 'You are a helpful and expert researcher answering questions about your scientific paper.', 'role': 'system'}, {'content': '[PAPERT-TITLE]\nGauging the environmental impact of urban farms and gardens\n[PAPER]\nAbstract There is a lack of data on resources used and food produced at urban farms. This hampers attempts to quantify the environmental impacts of urban agriculture or craft policies for sustainable food production in cities. To address this gap, we used a citizen science approach to collect data from 72 urban agriculture sites, representing three types of spaces (urban farms, collective gardens, individual gardens), in five countries (France, Germany, Poland, United Kingdom, and United States). We answered three key questions about urban agriculture with this unprecedented dataset: (1) What are its land, water, nutrient, and energy demands? (2) How productive is it relative to conventional agriculture and across types of farms? and (3) What are its contributions to

In [14]:
train_journalist_dataset = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-preprocessed-for-llama3/train_journalist_ds/')

In [25]:
#print(train_journalist_dataset['train']['prompt'][20])
[x[0]['content'] for x in train_journalist_dataset['train']['completion'][:50]]

['Journalist:  You mentioned that the study used a citizen science approach to collect data from 72 sites across five countries. Can you elaborate on how this approach was implemented and what key findings emerged regarding resource use and yields?',
 'Journalist:  The study also noted that farm type and management style were significant factors in resource use and yields. Could you explain how collective versus individual management influenced these outcomes?',
 'Journalist:  The study found that urban farms contributed significantly to local biodiversity, with an average of 20 different crops per farm. How does this diversity compare to conventional agriculture, and what are the implications for urban food systems?',
 'Journalist:  Finally, the study emphasizes the need for policymakers to support sustainable urban agriculture. What specific recommendations does the research offer for policymakers, and how can they work with urban farmers to implement these changes?',
 'Journalist:  